# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [4]:
#Dados de autenticação do twitter:
# consumer key = Kcq6B8XFQroCqT2dh7FvcWt0H 
# consumet secret = HwLL5rVZgkk5owuo662LA57o5IFVVsqZHM3LbzU4SMCnUT1rxm
# acess token = 905107777705652224-fA5Y1kkycCDfVR2xdAkopJktjRyVebA
# token secret = gNQZEXW7rxrk2fhXBhZBYbFBJKq7T4OLhX9OOHbjSZouJ

#Coloque aqui o identificador da conta no twitter: @annunzicate

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [ ]:
#Produto escolhido:
produto = 'Corote'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()
    
    

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [5]:
dados = pd.read_excel("Corote.xlsx")

del dados["Unnamed: 1"]
dados.columns = ["Treinamento","Classificacao"]

dados['Treinamento'] = dados['Treinamento'].str.replace("@"," ")
dados['Treinamento'] = dados['Treinamento'].str.replace("/"," ")
dados['Treinamento'] = dados['Treinamento'].str.replace("_"," ")
dados['Treinamento'] = dados['Treinamento'].str.replace(","," ")
dados['Treinamento'] = dados['Treinamento'].str.replace("."," ")
dados['Treinamento'] = dados['Treinamento'].str.replace("\n"," ")
dados['Treinamento'] = dados['Treinamento'].str.replace(":"," ")
dados['Treinamento'] = dados['Treinamento'].str.replace('"',"")
dados['Treinamento'] = dados['Treinamento'].str.replace('rt',"")
dados['Treinamento'] = dados['Treinamento'].str.replace('  '," ")
dados['Treinamento'] = dados['Treinamento'].str.replace('   '," ")
dados['Treinamento'] = dados['Treinamento'].str.replace('#'," ")

dados_irrelevante = dados[dados.Classificacao == 'IRRELEVANTE']
dados_relevante = dados[dados.Classificacao == 'RELEVANTE']


   

In [6]:
dados_irrelevante = pd.read_excel("Irrelevante.xlsx")
dados_relevante = pd.read_excel('Relevante.xlsx')

dados_irrelevante['Treinamento'] = dados_irrelevante['Treinamento'].map(lambda x: x.lstrip(' '))
dados_relevante['Treinamento'] = dados_relevante['Treinamento'].map(lambda x: x.lstrip(' '))




In [7]:
dados_relevante

,Treinamento,Classificacao
1,me chama pra beber corote,RELEVANTE
2,pqtif okumurajaque gabinogueira 12 bolsonibrun...,RELEVANTE
3,diegoopel quando isso? pior q se for pra ficar...,RELEVANTE
4,itsmealinep nao to falando mal de corote to fa...,RELEVANTE
5,não é possível que eu esqueci metade do role p...,RELEVANTE
6,jennidospeitos vamosss fuma um e toma uns coro...,RELEVANTE
7,paulabrgss quaa tem open de corote hmmm,RELEVANTE
8,se a vida te der um limão troque por um corote...,RELEVANTE
9,gmeelo9 nem cuo paga um corote,RELEVANTE
10,um corote na mao e mil ideias na cabeça,RELEVANTE


In [8]:
I = []
R = []
for l in dados_irrelevante.Treinamento:   
    partes = l.split()
    I.extend(partes)
    
for l in dados_relevante.Treinamento:   
    partes = l.split()
    R.extend(partes)
    

In [9]:
#NUMERO TOTAL DE PALAVRAS - IRRELEVANTES
lista_coluna_I = pd.Series(I)
len(lista_coluna_I)



2596

In [10]:

#NUMERO TOTAL DE PALAVRAS - RELEVANTES
lista_coluna_R = pd.Series(R)
len(lista_coluna_R)




1266

In [11]:
#QUANTIDADE QUE CADA PALAVRA APARECE - IRRELEVANTE
vcI = lista_coluna_I.value_counts()
X = []
for x in lista_coluna_I:
    if not x in X:
        X.append(x)
for i in X:
    a = vcI[i]
    #print("Essa palavra'{0}' possui {1} repetições".format(i,a))
    

In [12]:

#QUANTIDADE QUE CADA PALAVRA APARECE - RELEVANTE
vcR = lista_coluna_R.value_counts()
X = []
for x in lista_coluna_R:
    if not x in X:
        X.append(x)
for i in X:
    a = vcR[i]
    #print("Essa palavra'{0}' possui {1} repetições".format(i,a))
    


In [13]:
T = []
for i in I:
    T.append(i)
for i in R:
    T.append(i)
    
lista_coluna_T = pd.Series(T)
vcT = lista_coluna_T.value_counts()
len(vcT)

1361

In [14]:
IF = []
RF = []
TF = []

for l in dados_irrelevante.Treinamento:   
    partes = l.split()
    IF.append(partes)
    TF.append(partes)
    
for l in dados_relevante.Treinamento:   
    partes = l.split()
    RF.append(partes)
    TF.append(partes)



In [15]:
# AI = x+1
AI = []
AR = []
y = len(lista_coluna_I)
w = len(vcT)

for i in TF:
    j = 0
    ri = 1
    rr = 1
    for k in i:
    #VERIFICANDO PARA IRRELEVANTE
        if not k in vcI:
            ai = 1
            ri = (ai/(y+w))*ri
                
        if k in vcI:    
            ai = vcI[k] + 1
            ri = (ai/(y+w))*ri
    AI.append(ri)
    
    for k in i:
    #VERIFICANDO PARA IRRELEVANTE
        if not k in vcR:
            ar = 1
            rr = (ar/(y+w))*rr
                
        if k in vcR:    
            ar = vcR[k] + 1
            rr = (ar/(y+w))*rr
    AR.append(rr)
    
    




In [16]:
FRASE = []

for l in dados_irrelevante.Treinamento:   
    
    FRASE.append(l)
    
for l in dados_relevante.Treinamento:   
    
    FRASE.append(l)
    
len(FRASE)

300

In [17]:
FINAL = []

for i in range(len(AR)):
    if AR[i] > AI[i]:
        FINAL.append('RELEVANTE')
        #print("A frase '{0}' é relevante".format(FRASE[i]))
    else:
        FINAL.append('IRRELEVANTE')
        #print("A frase '{0}' é irrelevante".format(FRASE[i]))


In [18]:
tabela = pd.Series(FINAL)

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [19]:
teste = pd.read_excel("Corote.xlsx", sheetname="Teste")
teste

del teste["Unnamed: 1"]

teste.columns = ["Teste",'Classificacao']

teste['Teste'] = teste['Teste'].str.replace("@"," ")
teste['Teste'] = teste['Teste'].str.replace("/"," ")
teste['Teste'] = teste['Teste'].str.replace("_"," ")
teste['Teste'] = teste['Teste'].str.replace(","," ")
teste['Teste'] = teste['Teste'].str.replace("."," ")
teste['Teste'] = teste['Teste'].str.replace("\n"," ")
teste['Teste'] = teste['Teste'].str.replace(":"," ")
teste['Teste'] = teste['Teste'].str.replace('"',"")
teste['Teste'] = teste['Teste'].str.replace('rt',"")
teste['Teste'] = teste['Teste'].str.replace('  '," ")
teste['Teste'] = teste['Teste'].str.replace('   '," ")
teste['Teste'] = teste['Teste'].str.replace('#'," ")


In [20]:
teste_irrelevante = teste[teste.Classificacao == 'IRRELEVANTE']
teste_relevante = teste[teste.Classificacao == 'RELEVANTE']

In [21]:
#teste_irrelevante.to_excel("Irrelevante_t.xlsx")
#teste_relevante.to_excel("Relevante_t.xlsx")

In [22]:
teste_irrelevante = pd.read_excel("Irrelevante_t.xlsx")
teste_relevante = pd.read_excel("Relevante_t.xlsx")
#teste_relevante

In [23]:
TI = []
TR = []
for l in teste_irrelevante.Teste:   
    partes = l.split()
    TI.extend(partes)
    
for l in teste_relevante.Teste:   
    partes = l.split()
    TR.extend(partes)

In [24]:
#NUMERO TOTAL DE PALAVRAS - IRRELEVANTES
lista_coluna_TI = pd.Series(TI)
len(lista_coluna_TI)

1956

In [25]:
#NUMERO TOTAL DE PALAVRAS - RELEVANTES
lista_coluna_TR = pd.Series(TR)
len(lista_coluna_TR)


501

In [26]:
#QUANTIDADE QUE CADA PALAVRA APARECE - IRRELEVANTE
vcTI = lista_coluna_TI.value_counts()
X = []
for x in lista_coluna_TI:
    if not x in X:
        X.append(x)
for i in X:
    a = vcTI[i]
    #print("Essa palavra'{0}' possui {1} repetições".format(i,a))

In [27]:
#QUANTIDADE QUE CADA PALAVRA APARECE - RELEVANTE
vcTR = lista_coluna_TR.value_counts()
X = []
for x in lista_coluna_TR:
    if not x in X:
        X.append(x)
for i in X:
    a = vcTR[i]
    #print("Essa palavra'{0}' possui {1} repetições".format(i,a))

In [28]:
T = []
for i in TI:
    T.append(i)
for i in TR:
    T.append(i)
    
lista_coluna_T = pd.Series(T)
vcT = lista_coluna_T.value_counts()
len(vcT)

979

In [29]:
IFt = []
RFt = []
TFt = []

for l in teste_irrelevante.Teste:   
    partes = l.split()
    IFt.append(partes)
    TFt.append(partes)
    
for l in teste_relevante.Teste:   
    partes = l.split()
    RFt.append(partes)
    TFt.append(partes)
    


In [30]:
# AI = x+1
AIt = []
ARt = []
y = len(lista_coluna_TI)
w = len(vcT)

for i in TFt:
    j = 0
    ri = 1
    rr = 1
    for k in i:
    #VERIFICANDO PARA IRRELEVANTE
        if not k in vcTI:
            ai = 1
            ri = (ai/(y+w))*ri
                
        if k in vcTI:    
            ai = vcTI[k] + 1
            ri = (ai/(y+w))*ri
    AIt.append(ri)
    
    for k in i:
    #VERIFICANDO PARA IRRELEVANTE
        if not k in vcTR:
            ar = 1
            rr = (ar/(y+w))*rr
                
        if k in vcTR:    
            ar = vcTR[k] + 1
            rr = (ar/(y+w))*rr
    ARt.append(rr)

In [31]:
FRASE_t = []

for l in teste_irrelevante.Teste:   
    
    FRASE_t.append(l)
    
for l in teste_relevante.Teste:   
    
    FRASE_t.append(l)
    
len(FRASE_t)


200

In [32]:
FINAL_t = []

for i in range(len(ARt)):
    if ARt[i] > AIt[i]:
        FINAL_t.append('RELEVANTE')
        #print("A frase '{0}' é relevante".format(FRASE_t[i]))
    else:
        FINAL_t.append('IRRELEVANTE')
        #print("A frase '{0}' é irrelevante".format(FRASE_t[i]))

In [33]:
tab_teste = pd.Series(FINAL_t)
tab_teste


0      IRRELEVANTE
1      IRRELEVANTE
2      IRRELEVANTE
3      IRRELEVANTE
4      IRRELEVANTE
5      IRRELEVANTE
6      IRRELEVANTE
7      IRRELEVANTE
8      IRRELEVANTE
9      IRRELEVANTE
10     IRRELEVANTE
11     IRRELEVANTE
12     IRRELEVANTE
13     IRRELEVANTE
14     IRRELEVANTE
15     IRRELEVANTE
16     IRRELEVANTE
17     IRRELEVANTE
18     IRRELEVANTE
19     IRRELEVANTE
20     IRRELEVANTE
21     IRRELEVANTE
22     IRRELEVANTE
23     IRRELEVANTE
24     IRRELEVANTE
25     IRRELEVANTE
26     IRRELEVANTE
27     IRRELEVANTE
28     IRRELEVANTE
29     IRRELEVANTE
          ...     
170    IRRELEVANTE
171      RELEVANTE
172      RELEVANTE
173    IRRELEVANTE
174      RELEVANTE
175      RELEVANTE
176    IRRELEVANTE
177    IRRELEVANTE
178    IRRELEVANTE
179    IRRELEVANTE
180    IRRELEVANTE
181    IRRELEVANTE
182    IRRELEVANTE
183    IRRELEVANTE
184      RELEVANTE
185    IRRELEVANTE
186    IRRELEVANTE
187    IRRELEVANTE
188    IRRELEVANTE
189    IRRELEVANTE
190    IRRELEVANTE
191    IRREL

AS FRASES FORAM SEPARADAS EM TABELAS DIFERENTES. UMA IRRELEVANTE E OUTRA RELEVANTE. 



PRIMEIRO  FOI DADO UM APPEND NA TABELA IRRELEVANTE E DEPOIS NA RELEVANTE.

AS PRIMEIRAS 153 FRASES FORAM CLASSIFICADAS (MANUALMENTE) COMO IRRELEVANTE E AS 47 RESTANTES COMO RELEVANTES

ASSIM, É NECESSÁRIO COMPARAR PRIMEIRO COM A TABELA IRRELEVANTE E DEPOIS COM A TABELA RELEVANTE.

In [36]:
#POSITIVO FALSO = MARCADO COMO RELEVANTE MAS É IRRELEVANTE
#POSITIVO VERDADIRO = MARCADO COMO RELEVANTE E É RELEVANTE
#NEGATIVO VERDADEIRO = MARCADO COMO IRRELEVANTE E É IRRELEVANTE
#NEGATIVO FALSO = MARCADO COMO IRRELEVANTE MAS É RELEVANTE

POSITIVO_FALSO = [] 
POSITIVO_VERDADEIRO = []  
NEGATIVO_VERDADEIRO = []
NEGATIVO_FALSO = []

for i in range(154):
    if tab_teste[i] == 'IRRELEVANTE':
        NEGATIVO_VERDADEIRO.append("A frase '{0}' é negativo verdadeiro".format(i))
    if tab_teste[i] == 'RELEVANTE':
        NEGATIVO_FALSO.append("A frase '{0}' é negativo falso".format(i))
for i in range(154,200):
    if tab_teste[i] == 'RELEVANTE':
        POSITIVO_VERDADEIRO.append("A frase '{0}' é positivo verdadeiro".format(i))
    if tab_teste[i] == 'IRRELEVANTE':
        POSITIVO_FALSO.append("A frase '{0}' é positivo falso".format(i))
    
#QUANTIDADE QUE CADA UM TEM

PF = len(POSITIVO_FALSO)
PV = len(POSITIVO_VERDADEIRO)
NF = len(NEGATIVO_FALSO)
NV = len(NEGATIVO_VERDADEIRO)
        

    

In [37]:
PF

36

In [38]:
PV

10

In [39]:
NF

0

In [40]:
NV

154

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


1)A porcentagem de medidas irrelevantes é muito maior do que a de relevantes, assim como quando classificado manualmento, o que é algo positivo do programa.

1)Todos os dados classificados como irrelevante realmente eram irrelevantes, o que é algo positivo do programa.

1)Aproximadamente 20% dos dados classificados manualente como relevantes foram classificados como irrelevante pelo programa.

2)As mensagens com sarcasmo e dupla negação não conseguem ser bem entendidas pelo programa, desta maneira gerando confusão na hora de ser classificada.

3)De forma geral o programa teve um bom desempenho. Os dados classificados manualmente tiveram um resultado muito similar quando classificado pelo programa.Apenas 2,5% dos dados receberam classificação pelo programa diferente de quando analisado manualmente. O que mostra um grande eficiência.


